<a href="https://colab.research.google.com/github/shivaprajapati34390-netizen/ML-project/blob/main/Netflix_Stock_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
today=datetime.date.today()

In [22]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
today=datetime.date.today()

start_date=today-timedelta(days=5000)
end_date=today

!pip install yfinance
import yfinance as yf

data = yf.download('NFLX', start=start_date, end=end_date)

# Reset index to make 'Date' a regular column from the DatetimeIndex
data.reset_index(inplace=True)

# Flatten the MultiIndex columns, keeping only the first level (metric names).
# E.g., ('Open', 'NFLX') becomes 'Open'
data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]

# Select and reorder the desired columns
data = data[["Date", "Open", "High", "Low", "Close", "Volume"]]

# Ensure correct data types
data['Date'] = pd.to_datetime(data['Date'])
for col in ["Open", "High", "Low", "Close", "Volume"]:
    data[col] = pd.to_numeric(data[col], errors='coerce')

print("DataFrame Information:")
data.info()
print("\nFirst 5 rows of the DataFrame:")
print(data.head())

/tmp/ipython-input-1878078908.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('NFLX', start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3441 entries, 0 to 3440
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    3441 non-null   datetime64[ns]
 1   Open    3441 non-null   float64       
 2   High    3441 non-null   float64       
 3   Low     3441 non-null   float64       
 4   Close   3441 non-null   float64       
 5   Volume  3441 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 161.4 KB

First 5 rows of the DataFrame:
        Date      Open      High       Low     Close     Volume
0 2012-06-13  0.894286  0.920571  0.887571  0.897571  179249000
1 2012-06-14  0.897000  0.902143  0.871714  0.895143  261373000
2 2012-06-15  0.903857  0.941857  0.896143  0.939857  330225000
3 2012-06-18  0.929429  0.956429  0.925571  0.953286  229978000
4 2012-06-19  0.955714  1.006857  0.950714  0.997571  277067000


In [25]:
import plotly.graph_objects as go
figure=go.Figure(data=[go.Candlestick(x=data['Date'], open=data["Open"], high=data["High"], low=data["Low"], close=data["Close"])])
figure.update_layout(title="NFLX Stock Price Analysis",xaxis_rangeslider_visible=False)
figure.show()

In [27]:
correlation=data.corr()
print(correlation['Close'].sort_values(ascending=False))

Close     1.000000
Low       0.999852
High      0.999837
Open      0.999629
Date      0.859844
Volume   -0.452620
Name: Close, dtype: float64


In [28]:
# Traning LSTM foe netflix stock price prediction

In [29]:
x=data[["Open","High","Low","Volume"]]
y=data["Close"]
x=x.to_numpy()
y=y.to_numpy()
y=y.reshape(-1,1)


In [30]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)

In [32]:
from keras.models import Model, Sequential
from keras.layers import Dense, LSTM
model=Sequential()
model.add(LSTM(128,return_sequences=True,input_shape=(xtrain.shape[1],1)))
model.add(LSTM(64,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 4, 128)         │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,619 (459.45 KB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain, ytrain, batch_size=1, epochs=30)

Epoch 1/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 201.3789
Epoch 2/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 3.4144
Epoch 3/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 2.9964
Epoch 4/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 3.0864
Epoch 5/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 4.6813
Epoch 6/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 2.4647
Epoch 7/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 2.7565
Epoch 8/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 6.5692
Epoch 9/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 1.7400
Epoch 10/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 3.2431
Epoch 11/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 2.7068
Epoch 12/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 2.4746
Epoch 13/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 1.7617
Epoch 14/30
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 1.8435
Epoch 15/30
2

In [35]:
import numpy as np
# feature= [Open, High, Low, Adj Close, Volume]
features = np.array([[401.970001, 427.700012, 398.200012, 20047500]])
model.predict(features)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 444ms/step


array([[101.33124]], dtype=float32)